In [1]:
import requests

# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
auth = requests.auth.HTTPBasicAuth('IYJp6J95H4FhIIVYokFINQ', 'c_QVAbN4hGOqOuR6JgifU5Ch-EESuQ')

# here we pass our login method (password), username, and password
data = {'grant_type': 'password',
        'username': 'BDCCTeamRot',
        'password': 'Teambuilding1rot'}

# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'Reddiment'}

# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token', auth=auth, data=data, headers=headers)

#print(res)

# convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']

# add authorization to our headers dictionary
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}

# while the token is valid (~2 hours) we just add headers=headers to our requests
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

<Response [200]>

In [2]:
res = requests.get("https://oauth.reddit.com/r/python/hot", headers=headers)

antwort = res.json()

import pathlib
pathlib.Path('data.json').write_bytes(res.content)


#print(antwort)  # let's see what we get


156862

In [3]:
res.json()['data']['children']

[{'kind': 't3',
  'data': {'approved_at_utc': None,
   'subreddit': 'Python',
   'selftext': "Tell /r/python what you're working on this week! You can be bragging, grousing, sharing your passion, or explaining your pain. Talk about your current project or your pet project; whatever you want to share.",
   'author_fullname': 't2_145f96',
   'saved': False,
   'mod_reason_title': None,
   'gilded': 0,
   'clicked': False,
   'title': "Sunday Daily Thread: What's everyone working on this week?",
   'link_flair_richtext': [{'e': 'text', 't': 'Daily Thread'}],
   'subreddit_name_prefixed': 'r/Python',
   'hidden': False,
   'pwls': 6,
   'link_flair_css_class': 'daily-thread',
   'downs': 0,
   'thumbnail_height': None,
   'top_awarded_type': None,
   'hide_score': False,
   'name': 't3_uuz7id',
   'quarantine': False,
   'link_flair_text_color': 'light',
   'upvote_ratio': 0.73,
   'author_flair_background_color': '#7289da',
   'subreddit_type': 'public',
   'ups': 5,
   'total_awards_rece

In [7]:
from datetime import datetime
from elasticsearch import Elasticsearch

In [40]:
ELASTIC_PASSWORD = "test"

In [41]:
client = Elasticsearch(
    "http://localhost:9200",
    basic_auth=("elastic", ELASTIC_PASSWORD)
)

In [42]:
client.info()

ObjectApiResponse({'name': '369cb5601880', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'UEaY9AblQ8ygTqQf5QmStQ', 'version': {'number': '8.2.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'b174af62e8dd9f4ac4d25875e9381ffe2b9282c5', 'build_date': '2022-04-20T10:35:10.180408517Z', 'build_snapshot': False, 'lucene_version': '9.1.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [53]:
doc = {
    'author': 'Daniel',
    'text': 'Interensting content...',
    'timestamp': datetime.now(),
}

In [63]:
resp = client.index(index="test", id=3, document=doc)

In [64]:
print(resp)

{'_index': 'test', '_id': '3', '_version': 3, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 3, '_primary_term': 1}


In [50]:
resp = client.get(index="test-index", id=1)
print(resp)

{'_index': 'test-index', '_id': '1', '_version': 1, '_seq_no': 0, '_primary_term': 1, 'found': True, '_source': {'author': 'Daniel Holl', 'text': 'Interensting content...', 'timestamp': '2022-05-24T14:50:33.302809'}}


In [60]:
resp = client.search(index="test-index", query={"match_all": {}})

print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print("%(timestamp)s %(author)s: %(text)s" % hit["_source"])

print(resp)

Got 2 Hits:
2022-05-24T14:50:33.302809 Daniel Holl: Interensting content...
2022-05-24T14:55:00.136956 Ardian: Interensting content...
{'took': 3, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 2, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'test-index', '_id': '1', '_score': 1.0, '_source': {'author': 'Daniel Holl', 'text': 'Interensting content...', 'timestamp': '2022-05-24T14:50:33.302809'}}, {'_index': 'test-index', '_id': '2', '_score': 1.0, '_source': {'author': 'Ardian', 'text': 'Interensting content...', 'timestamp': '2022-05-24T14:55:00.136956'}}]}}


In [35]:
client.get(index='test-index',id=2)

ObjectApiResponse({'_index': 'test-index', '_id': '2', '_version': 1, '_seq_no': 1, '_primary_term': 1, 'found': True, '_source': {'author': 'Ardian', 'text': 'Interensting content...', 'timestamp': '2022-05-24T14:55:00.136956'}})

In [65]:
client.count(index='test-index')

ObjectApiResponse({'count': 2, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})